In [1]:
import requests
import sys
import os
import logging
sys.path.append("..")
from utils import load_azd_env, set_azure_ai_project_connection_string, setup_logging, get_file_paths

## Set environment variables

In [2]:
load_azd_env()
set_azure_ai_project_connection_string()

## Setup logging

In [3]:
enbale_logging = False

if enbale_logging:
    setup_logging()

## Test if FastAPI server is operational

In [4]:
project = requests.post(
    url = "http://127.0.0.1:8000/echo",
    json = {
        "msg": "Hello, World!"
    }
    )
print(project.json())

{'echo_msg': 'Echo: Hello, World!'}


## Create Azure AI Foundry project client

In [5]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

logging.info('create_project_client trigger function started processing a request.')

project_connection_string = os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING")

credential = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string,
    credential=credential
    )

## Create OpenAI client and call chat completion

In [6]:
openai = project_client.inference.get_azure_openai_client(api_version="2024-06-01")
response = openai.chat.completions.create(
    model=os.environ["AZURE_AI_DEPLOYMENT_MODEL"],
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant"},
        {"role": "user", "content": "Write me a poem about flowers"},
    ]
)
print(response.choices[0].message.content)

In meadows where the sunlight spills,
Where breezes hum and nature thrills,
The flowers bloom in colors bright,
A tapestry of pure delight.

The daisies nod with faces fair,
As if to greet the morning air,
While roses blush with crimson hue,
Their petals kissed by morning dew.

Bluebells chime in quiet glades,
Their delicate, cerulean shades,
And tulips stand in proud array,
Like soldiers on a springtime day.

The lavender, with fragrance sweet,
Beguiles the senses, oh so fleet,
While sunflowers with golden crowns,
Reflect the joy that summer found.

Hyacinths in clusters bloom,
Filling hearts with sweet perfume,
In colors rich and subtle too,
A spectrum dancing 'neath the blue.

And orchids, rarest of the rare,
With elegance beyond compare,
Unfold their secrets to the sky,
A mystery for every eye.

So many blooms, each one unique,
A language only they can speak,
In gardens wild or tended close,
They share a beauty we repose.

In petals soft, in colors riot,
In stillness deep or buzzin

In [7]:
"""
DESCRIPTION:
    This sample demonstrates how to use agent operations to add files to an existing vector store and perform search from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_vector_store_batch_file_search.py

    Before running the sample:

    pip install azure-ai-projects azure-identity

    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.
"""

from azure.ai.projects.models import FileSearchTool, FilePurpose

files = get_file_paths(data_folder = "../../data")

with project_client:

    # Create a vector store with no file and wait for it to be processed
    vector_store = project_client.agents.create_vector_store_and_poll(data_sources=[], name="sample_vector_store")
    print(f"Created vector store, vector store ID: {vector_store.id}")

    # Upload a file and wait for it to be processed
    file_ids = []
    for file in files:
        file = project_client.agents.upload_file_and_poll(file_path=file, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file, file ID: {file.id}")
        file_ids.append(file.id)

    # Add the file to the vector store or you can supply file ids in the vector store creation
    vector_store_file_batch = project_client.agents.create_vector_store_file_batch_and_poll(
        vector_store_id=vector_store.id, file_ids=file_ids
    )
    print(f"Created vector store file batch, vector store file batch ID: {vector_store_file_batch.id}")

    # Create a file search tool
    # [START create_agent_with_tools_and_tool_resources]
    file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

    # Notices that FileSearchTool as tool and tool_resources must be added or the assistant unable to search the file
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_AI_DEPLOYMENT_MODEL"],
        name="my-assistant",
        instructions="You are helpful assistant and can search information from uploaded files",
        tools=file_search_tool.definitions,
        tool_resources=file_search_tool.resources,
    )
    # [END create_agent_with_tools_and_tool_resources]
    print(f"Created agent, agent ID: {agent.id}")

    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content="How long does it take to prepare Christmas Pudding?"
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, run ID: {run.id}")

    file_search_tool.remove_vector_store(vector_store.id)
    print(f"Removed vector store from file search, vector store ID: {vector_store.id}")

    project_client.agents.update_agent(
        assistant_id=agent.id, tools=file_search_tool.definitions, tool_resources=file_search_tool.resources
    )
    print(f"Updated agent, agent ID: {agent.id}")

    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content="How long does it take to prepare Christmas Pudding?"
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, run ID: {run.id}")

    project_client.agents.delete_vector_store(vector_store.id)
    print("Deleted vector store")

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")


Created vector store, vector store ID: vs_9CCmnIrSEAfFoKqLZ2JuYRNW
Uploaded file, file ID: assistant-MRMCfHqlatBnZEh1qEzbUdjn
Uploaded file, file ID: assistant-ZYsGHZSewXYAGNdzVdXAKubh
Uploaded file, file ID: assistant-dvCdeDk8Zyscq8BfzPIjcoqk
Created vector store file batch, vector store file batch ID: vsfb_139a05cc60514ca6a0e0777899cbc02e
Created agent, agent ID: asst_1roJsYXx7HQRVb0Jg67lCGhE
Created thread, thread ID: thread_qhqHxxDZeZs95aOER4XbCdQM
Created message, message ID: msg_XhVpMPdRiRwqvmIaTTfZ6gGn
Created run, run ID: run_NvqiQGUZL74NgbjhFc668Oeq
Removed vector store from file search, vector store ID: vs_9CCmnIrSEAfFoKqLZ2JuYRNW
Updated agent, agent ID: asst_1roJsYXx7HQRVb0Jg67lCGhE
Created thread, thread ID: thread_p6P6wakLrCUUbQx2slGstQu8
Created message, message ID: msg_My6NNO0P5MZBjUBzJgxwCbht
Created run, run ID: run_wqp6EciASa5DZVUFmFgFLASC
Deleted vector store
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_My6NNO0P5MZBjUBzJgxwCbht', 'object': 'thread